In [1]:
import os

In [2]:
%pwd

'z:\\Data Science Projects\\Red wine ML -Project\\Red-wine-ML-project\\research'

In [3]:
# for changing the directory
os.chdir("../")

In [4]:
%pwd

'z:\\Data Science Projects\\Red wine ML -Project\\Red-wine-ML-project'

In [5]:
# Creating a entity
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True) # It will take only these
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [8]:
from wineQuality.constants import *
from wineQuality.utils.common import read_yaml, create_directories

In [9]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        
        create_directories([self.config.artifacts_root])
        
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        
        create_directories([config.root_dir])
        
        data_ingestion_config = DataIngestionConfig(
            root_dir= config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )
        
        return data_ingestion_config

In [10]:
import os
import urllib.request as request
import zipfile
from wineQuality import logger
from wineQuality.utils.common import get_size

In [12]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
        
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename=self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}") 
            
            
    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)               

In [13]:
# Creating a pipeline

try:
    config =ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-04-09 20:28:34,118:INFO:common:yaml file: config\config.yaml loaded successfully]
[2024-04-09 20:28:34,118:INFO:common:yaml file: params.yaml loaded successfully]
[2024-04-09 20:28:34,126:INFO:common:yaml file: schema.yaml loaded successfully]
[2024-04-09 20:28:34,136:INFO:common:create a directory at: artifacts]
[2024-04-09 20:28:34,136:INFO:common:create a directory at: artifacts/data_ingestion]
[2024-04-09 20:28:35,145:INFO:3949792308:artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 26148
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "f823ac36d764516b7832d5e153900a57ef608b1733000552b70140147a705680"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: C968:1AE1CD:41DC79:547B28:6615578F
Accept-Ranges: bytes
Date: Tue, 09 Apr 2024 14:58: